In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime

In [0]:
df_reservas_ota = (
    spark.read.format("delta").load("/Volumes/workspace/default/raw/tb_reservas_ota")
    .select(
        F.col("reserva_ota_id"),
        F.col("reserva_id"),
        F.col("canal"),
        F.col("comissao_percent"),
        F.col("data_reserva")
    )
    .withColumn("data_reserva", F.to_date(F.col("data_reserva")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_reservas_ota.limit(5))

In [0]:
catalog_path = "/Volumes/workspace/default/trusted/tb_reservas_ota"

if DeltaTable.isDeltaTable(spark, catalog_path):
    delta_table = DeltaTable.forPath(spark, catalog_path)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_reservas_ota.alias("source"),
        "target.reserva_ota_id = source.reserva_ota_id"  # chave única
    ).whenMatchedUpdate(
        set = {
            # Atualiza todas as colunas exceto update_date
            **{col: f"source.{col}" for col in df_reservas_ota.columns if col != "update_date"},
            # Atualiza a update_date com a data atual
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    df_reservas_ota.write.format("delta").save(catalog_path)